This notebook was automatically generated from your MAST-ML run so you can recreate the
plots. Some things are a bit different from the usual way of creating plots - we are
using the [object oriented
interface](https://matplotlib.org/tutorials/introductory/lifecycle.html) instead of
pyplot to create the `fig` and `ax` instances.


In [ ]:
"""
This module contains a collection of functions which make plots (saved as png files) using matplotlib, generated from
some model fits and cross-validation evaluation within a MAST-ML run.

This module also contains a method to create python notebooks containing plotted data and the relevant source code from
this module, to enable the user to make their own modifications to the created plots in a straightforward way (useful for
tweaking plots for a presentation or publication).
"""

import math
import os
import pandas as pd
import itertools
import warnings
import logging
from collections import Iterable
from os.path import join
from collections import OrderedDict

# Ignore the harmless warning about the gelsd driver on mac.
warnings.filterwarnings(action="ignore", module="scipy",
                        message="^internal gelsd")
# Ignore matplotlib deprecation warning (set as all warnings for now)
warnings.filterwarnings(action="ignore")

import numpy as np
from sklearn.metrics import confusion_matrix, roc_curve, auc, precision_recall_curve

import matplotlib
from matplotlib import pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.figure import Figure, figaspect
from matplotlib.animation import FuncAnimation
from matplotlib.font_manager import FontProperties
import matplotlib.mlab as mlab
from scipy.stats import gaussian_kde
from mpl_toolkits.axes_grid1.inset_locator import mark_inset
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes

# Needed imports for ipynb_maker
from mastml.utils import nice_range
from mastml.metrics import nice_names

import inspect
import textwrap
from pandas import DataFrame, Series

import nbformat

from functools import wraps

matplotlib.rc('font', size=18, family='sans-serif') # set all font to bigger
matplotlib.rc('figure', autolayout=True) # turn on autolayout

# adding dpi as a constant global so it can be changed later
DPI = 250

log = logging.getLogger() # only used inside ipynb_maker I guess



In [ ]:
def stat_to_string(name, value):
    """
    Method that converts a metric object into a string for displaying on a plot

    Args:

        name: (str), long name of a stat metric or quantity

        value: (float), value of the metric or quantity

    Return:

        (str), a string of the metric name, adjusted to look nicer for inclusion on a plot

    """

    " Stringifies the name value pair for display within a plot "
    if name in nice_names:
        name = nice_names[name]
    else:
        name = name.replace('_', ' ')

    # has a name only
    if not value:
        return name
    # has a mean and std
    if isinstance(value, tuple):
        mean, std = value
        return f'{name}:' + '\n\t' + f'{mean:.3f}' + r'$\pm$' + f'{std:.3f}'
    # has a name and value only
    if isinstance(value, int) or (isinstance(value, float) and value%1 == 0):
        return f'{name}: {int(value)}'
    if isinstance(value, float):
        return f'{name}: {value:.3f}'
    return f'{name}: {value}' # probably a string


def plot_stats(fig, stats, x_align=0.65, y_align=0.90, font_dict=dict(), fontsize=14):
    """
    Method that prints stats onto the plot. Goes off screen if they are too long or too many in number.

    Args:

        fig: (matplotlib figure object), a matplotlib figure object

        stats: (dict), dict of statistics to be included with a plot

        x_align: (float), float denoting x position of where to align display of stats on a plot

        y_align: (float), float denoting y position of where to align display of stats on a plot

        font_dict: (dict), dict of matplotlib font options to alter display of stats on plot

        fontsize: (int), the fontsize of stats to display on plot

    Returns:

        None

    """

    stat_str = '\n'.join(stat_to_string(name, value)
                           for name,value in stats.items())

    fig.text(x_align, y_align, stat_str,
             verticalalignment='top', wrap=True, fontdict=font_dict, fontproperties=FontProperties(size=fontsize))


def make_fig_ax(aspect_ratio=0.5, x_align=0.65, left=0.10):
    """
    Method to make matplotlib figure and axes objects. Using Object Oriented interface from https://matplotlib.org/gallery/api/agg_oo_sgskip.html

    Args:

        aspect_ratio: (float), aspect ratio for figure and axes creation

        x_align: (float), x position to draw edge of figure. Needed so can display stats alongside plot

        left: (float), the leftmost position to draw edge of figure

    Returns:

        fig: (matplotlib fig object), a matplotlib figure object with the specified aspect ratio

        ax: (matplotlib ax object), a matplotlib axes object with the specified aspect ratio

    """
    # Set image aspect ratio:
    w, h = figaspect(aspect_ratio)
    fig = Figure(figsize=(w,h))
    FigureCanvas(fig)

    # Set custom positioning, see this guide for more details:
    # https://python4astronomers.github.io/plotting/advanced.html
    #left   = 0.10
    bottom = 0.15
    right  = 0.01
    top    = 0.05
    width = x_align - left - right
    height = 1 - bottom - top
    ax = fig.add_axes((left, bottom, width, height), frameon=True)
    fig.set_tight_layout(False)
    
    return fig, ax


In [ ]:
def plot_predicted_vs_true(train_quad, test_quad, outdir, label):
    """
    Method to create a parity plot (predicted vs. true values)

    Args:

        train_quad: (tuple), tuple containing 4 numpy arrays: true training y data, predicted training y data,

        training metric data, and groups used in training

        test_quad: (tuple), tuple containing 4 numpy arrays: true test y data, predicted test y data,

        testing metric data, and groups used in testing

        outdir: (str), path to save plots to

        label: (str), label used for axis labeling

    Returns:

        None

    """
    filenames = list()
    y_train_true, y_train_pred, train_metrics, train_groups = train_quad
    y_test_true, y_test_pred, test_metrics, test_groups = test_quad

    # make diagonal line from absolute min to absolute max of any data point
    # using round because Ryan did - but won't that ruin small numbers??? TODO this
    max1 = max(y_train_true.max(), y_train_pred.max(),
               y_test_true.max(), y_test_pred.max())
    min1 = min(y_train_true.min(), y_train_pred.min(),
               y_test_true.min(), y_test_pred.min())
    max1 = round(float(max1), rounder(max1-min1))
    min1 = round(float(min1), rounder(max1-min1))
    for y_true, y_pred, stats, groups, title_addon in \
            (train_quad+('train',), test_quad+('test',)):

        # make fig and ax, use x_align when placing text so things don't overlap
        x_align=0.64
        fig, ax = make_fig_ax(x_align=x_align)

        # set tick labels
        # notice that we use the same max and min for all three. Don't
        # calculate those inside the loop, because all the should be on the same scale and axis
        _set_tick_labels(ax, max1, min1)

        # plot diagonal line
        ax.plot([min1, max1], [min1, max1], 'k--', lw=2, zorder=1)

        # do the actual plotting
        if groups is None:
            ax.scatter(y_true, y_pred, color='blue', edgecolors='black', s=100, zorder=2, alpha=0.7)
        else:
            handles = dict()
            unique_groups = np.unique(np.concatenate((train_groups, test_groups), axis=0))
            log.debug(' '*12 + 'unique groups: ' +str(list(unique_groups)))
            colors = ['blue', 'red', 'green', 'purple', 'orange', 'black']
            markers = ['o', 'v', '^', 's', 'p', 'h', 'D', '*', 'X', '<', '>', 'P']
            colorcount = markercount = 0
            for groupcount, group in enumerate(unique_groups):
                mask = groups == group
                log.debug(' '*12 + f'{group} group_percent = {np.count_nonzero(mask) / len(groups)}')
                handles[group] = ax.scatter(y_true[mask], y_pred[mask], label=group, color=colors[colorcount],
                                            marker=markers[markercount], s=100, alpha=0.7)
                colorcount += 1
                if colorcount % len(colors) == 0:
                    markercount += 1
                    colorcount = 0
            ax.legend(handles.values(), handles.keys(), loc='lower right', fontsize=12)

        # set axis labels
        ax.set_xlabel('True '+label, fontsize=16)
        ax.set_ylabel('Predicted '+label, fontsize=16)

        plot_stats(fig, stats, x_align=x_align, y_align=0.90)

        filename = 'predicted_vs_true_'+ title_addon + '.png'
        filenames.append(filename)
        fig.savefig(join(outdir, filename), dpi=DPI, bbox_inches='tight')

    return filenames


In [ ]:
from numpy import array
from collections import OrderedDict
from io import StringIO
train_quad = (array([177.28302038, 150.37102595, 202.0924573 , ...,   0.        ,
        47.20790294,  23.809066  ]), array([8.86415225e+01, 7.51855248e+01, 1.01046242e+02, ...,
       1.25861019e-06, 2.36039515e+01, 1.19045330e+01]), OrderedDict([('R2', 0.9949680293013485), ('root_mean_squared_error', 69.77936751243602), ('mean_absolute_error', 52.4481326229368), ('rmse_over_stdev', 0.7554385970649258)]), None)
test_quad = (array([218.80057061, 296.50121028, 209.63441278, 388.39517941,
       344.79927541,   0.        , 259.72112607,  64.76678569,
       120.56546405, 156.51523   , 110.503751  ,  95.94694389,
       241.9912159 , 185.18233461, 304.21609798,  67.39021938,
       178.59094562, 149.58971125, 123.82634292, 127.30671062,
        85.32116056,  97.75371527,  88.06968527,  89.46412177,
       170.11252586, 188.72532304,   0.        , 144.36770295,
       190.00207148, 166.69938886, 158.79174898, 350.0195967 ,
       306.66689715, 185.72882631, 116.8908945 ,  93.38585243,
       137.48865912, 127.740217  ,  91.15693652,  83.80925664,
        63.04573585,  38.74908992,   0.        ,  76.60506372,
       163.18574922,  95.14422461,  98.51156618,  70.97342159,
        86.04966315,  89.72129861, 515.80533047, 132.8149808 ,
       227.4912848 , 136.31607677, 201.81756512, 134.18069341,
        71.44563177, 347.3831    ,  57.32200585,  69.28696139,
         9.79676617,  56.15299875,   0.        ,  50.0993    ,
        27.31353641, 141.66475147, 197.42137823,  40.02993181,
        26.88964985,   0.        ,  56.47995687,   1.16763996,
         0.        ,  10.5881855 ,   0.        ,  29.91808225,
         0.        , 127.82578903, 288.48656282, 119.77883703,
       253.56044597, 219.40213351,  50.02545233,  75.15595576,
        33.13845536,   0.        ,   0.        ,  47.01813711,
        55.71532531,  16.66840972,   0.        ,   0.        ,
         0.        ,   0.        ,   3.60810735,   0.        ,
         0.        ,   0.        ,   0.        ,  19.90147862,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,  85.96097361,  75.38976676,   0.        ,
        38.14616569,   0.        ,  56.75173357,  43.12521542,
       210.80892907, 232.20920857, 189.64450984, 140.19831868,
       194.26853657,  69.26938771,  87.36071311,   0.        ,
        89.91354036,   8.97062587,  50.8263675 , 141.47054363,
        62.94422008,  79.23257647, 118.54297288,  22.18427375,
        39.83120675,  47.61927987,  57.49950031, 117.71448387,
       157.59559963, 132.10902428,  65.29481037, 123.75401639,
        26.83259762,  85.23364294, 187.39104625, 175.57959645,
         0.        ,  87.78436313, 285.94816437, 109.03341082,
        69.47351553, 232.45401086,  44.54144834,   0.        ,
        52.02867683, 207.51571353,  77.8684422 ,  47.36934588,
         0.        , 197.24625694, 211.4607162 , 202.28394837,
       199.68456861, 128.44886562, 211.47682381,  49.69176246,
         0.        ,  57.38221378,  36.16956355,  40.8070147 ,
         0.        ,  32.97595651,  28.05755061,   0.        ,
         0.        ,   0.        ,  21.5174731 ,   0.        ,
        52.56517063,  47.14197337,  48.10016777, 155.46507812,
       204.09999347,  91.38113959, 135.99188516,  26.27533825,
        82.66465971, 110.05200605,  50.298441  ,  37.03306019,
        39.54298544,  41.29569684,   0.        ,  52.0549201 ,
         0.        ,   0.        ,   0.        , 141.4711659 ,
       120.59661522,  94.30190586,  42.90406645,  17.67339581,
        33.67375834,  42.74924617,  47.24337609, 103.2663285 ,
        44.01037784,  14.00114963,  19.80860463,  17.71877028,
         5.43550612,  68.90928714, 125.21536969, 178.13772044,
        32.611025  ,  57.14438148,   5.20671817, 159.11770491,
       179.41288932, 245.69243888, 250.25203832, 260.12973722,
       132.8719224 , 168.86730174, 361.57977723, 367.1383674 ,
       366.50308111, 276.00692815,  90.31694834, 106.514784  ,
       110.13931779, 142.29581759, 241.66495314, 181.29528486,
       118.10326639, 124.47790803, 122.77257944, 149.94833269,
       189.88100424, 124.31412396, 170.62748097, 218.44261626,
       140.40595738,  87.04599469, 129.34261745, 101.76810798,
       156.13605672, 177.75216791, 351.81750188, 367.1832279 ,
       227.87867171, 201.32142318, 233.8102357 ,  76.49603762,
       129.31911366,  70.43789669, 149.64777796,  91.28475717,
       166.21281277, 252.16135397, 296.09591312, 141.92110132,
       159.4014052 , 234.968862  , 253.01986447, 353.34957447,
       315.51487515, 332.21213694, 252.00105048, 258.6400468 ,
       247.47424665, 252.61149593, 223.30782654, 318.43464993,
        68.40090273,  55.94984366, 148.87388686, 123.03536001,
       223.62476104,  59.52415623, 158.20291287,  54.1587672 ,
        53.12824467, 202.52553418, 312.11562515, 299.11257829,
       295.58204257, 845.33730031, 950.22790588, 151.15513036,
       105.93237224,  84.20838368,  51.98836201,  46.57820663,
       105.83615073,  95.42710656, 108.31218227, 125.30956249,
       122.02858221, 106.91311671, 156.51808854,  90.4495542 ,
        83.53425191,  39.04655223,  68.98007677,  36.00668628,
        18.90387551,  12.23389363,  17.43031391,  93.12084659,
        32.64732478, 112.90930286, 125.17193124,  30.53286251,
       131.67141422,  75.98358167,  58.54469733,  27.21757246,
        58.37804795,  42.90435228,  77.59338173, 173.73002813,
       234.93787541, 187.84859075, 174.90906864, 211.33526584,
       160.45672687, 146.55208232, 187.20717792, 165.12741257,
       179.71035609, 184.78773175,  54.91523627,  33.28049783,
       159.40354586, 137.47965258, 106.74575874, 113.88620361,
       102.77542034, 104.37297146, 296.63023043, 113.09210788,
       137.58344095, 397.23996543, 237.58337883, 111.06672169,
       117.14051442, 113.50149873, 110.86471789, 298.24570743,
       134.35498302,  37.14261864, 118.9471195 ,  22.37923032,
        25.29305299,  95.6649845 ,  98.84644072,  28.44172814,
        52.87341873, 179.36951362,  38.13605333,  95.53083323,
         0.        ,  49.53238936,  66.26244234,   0.        ,
        64.14092369,   4.75557533,  93.79066576,   0.        ,
        54.15149222,  45.40436553, 121.97489618,   0.        ,
        74.35889627,  39.79517364,   0.        ,  43.46433055,
         0.        ,  12.40117081,   0.        ,  28.40700128,
        34.10036925,  27.11965388,  31.76337063,  37.78468525,
        29.90345073,  14.17874738,   0.        ,   0.        ,
       184.38740476,  36.096681  ,  97.23119343,  42.38549807,
        33.9806022 ,  40.82068744,  40.23930643,   0.        ,
       103.73141473, 123.84952014,  87.26704216,  82.62024503,
        73.30236834,  67.66092594,  71.76198016, 110.66477369,
        84.87592237,  41.13790728,  30.59603878,  80.17227524,
        29.23073416,  24.51250794,  25.95621478,  94.00578421,
        80.15612571,  70.04323975,  17.00030381,  45.40761163,
        45.06238163,  45.32728206,  92.80824162,   5.974325  ,
       121.7771504 ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   1.58851259,
         8.66887169,   0.        ,   0.        ,  36.55454693,
         0.        ]), array([ 0.00000000e+000,  4.92516655e-073,  0.00000000e+000,
        0.00000000e+000,  0.00000000e+000,  0.00000000e+000,
        0.00000000e+000,  3.56258670e-028,  9.64563318e-028,
        1.25131300e-132,  1.93841233e-132,  2.16669030e-231,
        7.09097464e-028,  2.36597089e-238,  3.03592254e-115,
        1.68120307e-027,  3.94752770e-027,  5.96436037e-192,
        4.89840557e-192,  1.05056155e-132,  2.40698312e-231,
        5.86743022e-192,  4.35093454e-192,  4.12007653e-192,
        6.87005898e-281,  8.42161517e-192,  5.22041378e-308,
        1.93017472e-010,  2.25516103e-130,  0.00000000e+000,
        0.00000000e+000,  7.41972821e-123,  0.00000000e+000,
        5.69420679e-032,  1.90685154e-252,  1.36820042e-032,
        1.07967237e-132,  1.59735846e-132,  0.00000000e+000,
        0.00000000e+000,  3.16813674e+001,  4.20480917e-014,
        1.20849218e-157,  1.59648651e-038,  1.60886977e-007,
        5.12641420e-027,  3.16365228e-025,  1.75387438e-103,
        2.35402083e+000,  1.77195334e+000,  1.42764636e-018,
        5.92561269e-091,  0.00000000e+000,  1.53022013e-010,
        4.69569710e-045,  7.49227516e+000,  1.31512880e-013,
        5.39575881e-040,  1.70281995e-038,  4.86408510e-024,
        0.00000000e+000,  2.98137223e-026,  3.23319374e-108,
        0.00000000e+000,  7.13890521e-039,  0.00000000e+000,
        3.87694809e-009,  4.21271129e-002,  1.64059738e-010,
        3.80676788e-044,  4.42365260e-003,  0.00000000e+000,
        0.00000000e+000,  0.00000000e+000,  0.00000000e+000,
        0.00000000e+000,  1.70934223e-005,  2.02691965e-010,
        0.00000000e+000,  1.08912435e+001,  4.64663445e-002,
        4.42833004e-009,  1.35251508e-317,  3.61381410e-234,
        3.32575071e-072,  0.00000000e+000,  0.00000000e+000,
        1.03560569e-001,  1.39933685e-008,  2.56891819e-084,
        0.00000000e+000,  7.38120149e-008,  8.33643449e-271,
        1.53068242e-238,  2.48672203e-084,  4.52271122e-097,
        7.10536500e-239,  1.01570194e-238,  2.69748985e-235,
        1.24881219e-242, -1.02293220e-256,  0.00000000e+000,
        0.00000000e+000, -1.93941905e-015, -3.03574336e-009,
        2.06043524e-004,  9.96131753e-072,  2.22288072e-031,
        2.49239196e-095,  0.00000000e+000,  4.50587078e-012,
        9.01557907e-003,  6.70779449e-084,  1.35637112e-094,
        1.11989778e-011,  8.59228483e-006,  1.14773458e-071,
        2.23444414e+000,  2.32373964e-095,  1.34945220e-241,
        2.05124155e-034,  0.00000000e+000,  2.41910045e-006,
        4.31746609e-009,  1.34939626e-027,  4.10518407e-048,
        3.68440656e-021,  1.61027419e-052,  1.44580073e-030,
        3.27548944e-012,  8.73414713e-003,  3.89569346e-006,
        0.00000000e+000,  4.55868298e-205,  0.00000000e+000,
        2.92138535e+000,  0.00000000e+000,  2.04434208e-026,
        4.20930396e-094,  2.39704330e-011,  0.00000000e+000,
        9.44792836e+000,  1.58341565e-024,  8.52268226e-023,
        0.00000000e+000,  0.00000000e+000,  0.00000000e+000,
        4.01317728e-017,  1.54884335e-113,  2.43180886e-027,
        5.79253867e-029,  3.25858152e-002,  1.14122705e-056,
        2.32545494e-013,  4.96709567e-038,  1.20457103e-001,
        2.02194097e-224,  1.19721968e-164,  3.26546290e-153,
        1.25417713e-101,  0.00000000e+000,  6.13489446e-008,
        1.02469067e+000,  1.29732983e-004,  0.00000000e+000,
        2.12816764e-020,  1.54604175e-008,  1.35876245e-008,
        0.00000000e+000,  0.00000000e+000,  2.68165322e-013,
        0.00000000e+000,  6.79018023e-008,  8.08665397e-004,
        5.13889286e+000,  1.83111272e-087,  2.23741546e-101,
        9.92080963e-008,  2.07090922e-076,  2.12017879e-025,
        7.05344742e-076,  4.48884416e-033,  0.00000000e+000,
        5.69754909e-102,  9.99740503e+000,  8.71030965e-002,
        8.14378017e-009,  7.57669787e-002,  0.00000000e+000,
        0.00000000e+000,  0.00000000e+000,  1.99618584e-007,
        5.44626115e-248,  5.14706978e-088,  6.97671932e-071,
        1.69963947e-009,  2.58197093e-157,  1.55272154e-069,
        2.84208486e-025,  9.20198492e-008,  1.38052896e+001,
        5.76328059e-039,  1.34547739e-020,  0.00000000e+000,
        0.00000000e+000,  1.68368267e-003,  2.86745508e-071,
        1.60804163e-031,  0.00000000e+000,  1.47553857e-088,
        3.29865207e-013,  4.05691358e-001,  3.24249184e-001,
        1.18172906e-064,  2.63728895e-043,  7.68841362e-015,
        3.45645532e-001,  3.77649539e-008,  8.03369001e-001,
        9.86527993e+000,  7.48023554e+000,  1.06741180e-003,
        2.14334652e+000,  1.76082854e-001,  1.88161533e-001,
        2.53352135e+000,  5.36558010e-001,  2.21316506e-005,
        3.14444056e+000,  9.25768261e-021,  1.09760211e-008,
        4.68118937e-001,  4.17575232e-008,  3.18495170e-001,
        3.30499873e-001,  5.62908337e-002,  1.57433611e-008,
        8.37657996e+000,  1.23474861e+001,  1.07130365e+001,
        6.80069980e-011,  3.89783277e-004,  7.58154540e-001,
        7.69435745e-001,  5.17668837e+000,  4.75000941e+000,
        5.68401488e+000,  1.86746244e+000,  1.45011146e+000,
        2.12051267e-001,  2.51029577e-001,  2.30233208e-008,
        4.42923324e-001,  1.86510952e-001,  8.23506191e-002,
        2.97544815e-001,  4.13263136e+000,  4.74604928e+000,
        5.06334133e-001,  7.19513271e-001,  6.98451758e-001,
        9.15339476e+000,  6.66032337e+000,  6.89315516e+000,
        4.85296321e+000,  7.37761370e-011,  4.23047162e-004,
        7.71818311e+000,  5.05274135e-021,  5.99010370e-009,
        3.86810962e-001,  2.32190384e-001,  5.66650972e-002,
        1.15986629e-001,  1.06695381e-001,  1.42429974e-008,
        4.56788441e+000,  3.91488369e-001,  6.89767697e-001,
        4.45794470e-001,  9.05465385e+000,  3.12000558e-010,
        6.97067524e-010,  7.31295247e-017,  1.01857545e-159,
        1.89605597e-070,  4.01823629e-017,  5.91148457e-160,
        2.74709464e-001,  7.50152642e-004,  8.57841866e-004,
        1.41264316e-002,  1.02473126e-003,  2.92635929e-001,
        1.17475848e-003,  7.36445440e-004,  7.19889024e-004,
        4.83662126e-103,  3.34537862e-045,  5.94840739e-006,
        2.73615895e-001,  1.58845228e-017,  3.78803850e-027,
        1.38042342e-045,  3.86932256e-017,  1.68088907e-109,
        1.03904612e-003,  4.42109549e-001,  9.92581409e-004,
        5.36678293e-004,  4.30208721e-001,  1.60822042e-004,
        3.32530972e-004,  2.29679230e-004,  4.40714115e-004,
        4.90144024e-015,  7.81974164e-051,  3.66353362e-003,
        5.26014307e-015,  9.66323854e-145,  7.45421963e-234,
        2.30008268e-103,  1.37517247e-002,  1.47734326e-003,
        4.68504945e+000,  1.27347381e-003,  0.00000000e+000,
       -1.35876359e-229,  0.00000000e+000,  3.31022099e+000,
        9.51054635e-004,  1.07139120e-003,  9.67947606e+000,
        9.96302446e+000,  2.01061789e-003,  8.70555215e-001,
        3.14610674e+000,  0.00000000e+000,  6.00189415e+000,
        1.20424659e-004,  4.45393297e+000,  1.13420701e-004,
        1.10161162e+001,  2.78007938e-004,  3.10974832e+000,
        1.01376731e+000,  1.34844425e-002,  9.12894314e-183,
        1.00745490e-080,  8.56302146e-003,  1.46798274e-012,
        2.13322183e-021,  9.62981222e-007,  2.60273026e-090,
        3.20506923e-092,  6.96431159e-024,  0.00000000e+000,
        7.68013723e-031,  3.25946924e-013, -8.93388274e-243,
        4.23227779e-007,  9.08703613e-003,  5.16865374e-078,
        0.00000000e+000,  0.00000000e+000,  0.00000000e+000,
        3.88863043e+000,  0.00000000e+000,  2.22897728e-086,
        8.14752766e-060,  3.50861261e-021,  5.30327474e-078,
        0.00000000e+000,  1.96894455e-003,  0.00000000e+000,
        2.32780833e-005,  3.58269641e-028,  4.61782348e-031,
        5.52039562e-011,  7.88446535e-018,  6.74008243e-001,
        3.07585248e-003,  0.00000000e+000,  0.00000000e+000,
        6.83244198e-080,  3.36024419e-026,  3.99231470e-012,
        1.17057177e-017,  7.34101178e-002,  7.94777692e-003,
        5.54696870e-002,  0.00000000e+000,  1.26048736e+000,
        1.37843784e-077,  1.17299060e-030,  0.00000000e+000,
        0.00000000e+000,  1.35543134e-052,  4.51933044e-023,
        3.04469478e+000,  1.18058308e-077,  4.60703029e-028,
        8.24054618e-018,  7.51878597e-063,  4.82107672e-009,
        6.51944826e-085,  4.33647088e-017,  5.48898771e-008,
        2.15553867e-025,  1.69081890e-080,  0.00000000e+000,
        6.85751162e-028,  6.67061916e-011,  1.31865743e-017,
        2.71866738e-006,  1.30644998e-052,  9.78739117e-007,
        0.00000000e+000,  0.00000000e+000,  0.00000000e+000,
        0.00000000e+000,  0.00000000e+000,  2.32020912e-063,
        1.79838875e-030,  9.49633148e-013,  0.00000000e+000,
        5.75215479e-003,  2.12448279e-003,  0.00000000e+000]), OrderedDict([('R2', 0.015244623965862902), ('root_mean_squared_error', 150.62316731930122), ('mean_absolute_error', 106.64152659908257), ('rmse_over_stdev', 1.6306618741548429)]), None)
outdir = '/home/nerve/Desktop/skunkworks/lane_schultz/hyper_parameters/test/DoNothing/DoNothing/KernelRidge/RepeatedKFold/split_5'
label = 'Energy above convex hull (meV/atom)'

In [ ]:
import pandas as pd
from IPython.display import Image, display

plot_paths = plot_predicted_vs_true(train_triple, test_triple, outdir)
for plot_path in plot_paths:
    display(Image(filename=plot_path))
